Note: This Notebook is used to continue to explore from LoanCleanUp2 post-presentation.

This notebook will begin the process of fixing error in recovery variable and completely removing Grace Period (issued) as part of Good or Bad

Extremely Messy and Needs to use functions to avoid Copy And Paste

In [2]:
import pandas as pd # the pd is by convention
import numpy as np # as is the np

import matplotlib.pyplot as plt
import seaborn as sns


import pickle

# To Plot matplotlib figures inline on the notebook
%matplotlib inline

# Importing Loans

In [3]:
loan_df = pd.read_csv("loan.csv", low_memory=False)

In [ ]:
Dictionary_df = pd.read_excel("LCDataDictionary.xlsx")

# Analysis of Data: NaN's, Variables Desired, And SAVE ME!

In [ ]:
def uniquecontents(df, col):
    print(df[col].unique())

def showcontents(df, col):
    print(df[col])

In [ ]:
uniquecontents(loan_df, "desc") #loan_df["desc"].unique()

In [ ]:
uniquecontents(loan_df, "earliest_cr_line")
uniquecontents(loan_df, "purpose")
uniquecontents(loan_df, "loan_status")
uniquecontents(loan_df, "title")
uniquecontents(loan_df, "tot_coll_amt")
uniquecontents(loan_df, "total_rec_int")


## Featured Engineering - Loans and Recovery

####  **Looking at columns and their entries in a statistical summary** 

In [56]:
loan_df.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

In [4]:
loan_df = loan_df[loan_df.loan_status != 'In Grace Period']
loan_df['good_loan'] = np.where((loan_df.loan_status == 'Fully Paid') |
                        (loan_df.loan_status == 'Current') | 
                        (loan_df.loan_status == 'Does not meet the credit policy. Status:Fully Paid'), 1, 0)

In [5]:
loan_df['recoveries'] = np.where((loan_df.recoveries != 0.00), 1, 0)

#### Percent of the data is absent > 60%?

In [6]:
check_null = loan_df.isnull().sum(axis=0).sort_values(ascending=False)/float(len(loan_df))
check_null[check_null > 0.6]

dti_joint                      0.999426
verification_status_joint      0.999423
annual_inc_joint               0.999423
il_util                        0.978956
mths_since_rcnt_il             0.976476
open_acc_6m                    0.975838
open_il_6m                     0.975838
open_il_12m                    0.975838
open_il_24m                    0.975838
total_bal_il                   0.975838
open_rv_12m                    0.975838
open_rv_24m                    0.975838
max_bal_bc                     0.975838
all_util                       0.975838
inq_last_12m                   0.975838
inq_fi                         0.975838
total_cu_tl                    0.975838
desc                           0.857550
mths_since_last_record         0.845603
mths_since_last_major_derog    0.750582
dtype: float64

#### Drop NaN Rows & Irrelevant Columns


In [7]:
loan_df.drop(check_null[check_null>0.5].index, axis=1, inplace=True) 
loan_df.dropna(axis=0, thresh=30, inplace=True)

In [8]:
#Delete columns that are not relevant or lacking in data
delete_me = ['policy_code', 'url', 'id', 'member_id', 'application_type', 'acc_now_delinq','emp_title', 'zip_code','title']
loan_df.drop(delete_me , axis=1, inplace=True) 

In [9]:
# strip months from 'term' and make it an int
 loan_df['term_num'] = loan_df['term'].str.split(' ').str[1]

#interest rate into a 
loan_df['int_rate'] = loan_df.int_rate.astype(float)/100.

# extract numbers from emp_length and fill missing values with the median
loan_df['emp_length'] = loan_df['emp_length'].str.extract('(\d+)').astype(float)
loan_df['emp_length'] = loan_df['emp_length'].fillna(loan_df.emp_length.median()) 
    #Maybe be problematic, so may just remove this if working with just 2017 data set


#Relook on what fails
##[0]#.str[1]

In [10]:
lack_of_data_idx = [x for x in loan_df.count() < 887379*0.25]
loan_df.drop(loan_df.columns[lack_of_data_idx], 1, inplace=True)

In [66]:
good_loan =  len(loan_df[(loan_df.loan_status == 'Fully Paid') |
                    (loan_df.loan_status == 'Current') | 
                    (loan_df.loan_status == 'Does not meet the credit policy. Status:Fully Paid')])
print ('Good/Bad Loan Ratio: %.2f%%'  % (good_loan/len(loan_df)*100))

Good/Bad Loan Ratio: 92.10%


In [11]:
check_null = loan_df.isnull().sum(axis=0).sort_values(ascending=False)/float(len(loan_df))
check_null[check_null>0.05]


next_pymnt_d        0.287100
tot_cur_bal         0.079682
tot_coll_amt        0.079682
total_rev_hi_lim    0.079682
dtype: float64

In [12]:
loan_df = loan_df.drop(columns = ['next_pymnt_d', 'last_pymnt_d'])
loan_df = loan_df.dropna() #remove rows where there is atlease one NAN or empty
loan_df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,good_loan
42535,27050.0,27050.0,27050.0,36 months,0.1099,885.46,B,B2,10.0,OWN,...,0.00,0,0.0,885.46,Dec-2015,0.0,0.0,114834.0,59900.0,1
42536,9750.0,9750.0,9750.0,36 months,0.1398,333.14,C,C1,1.0,RENT,...,0.00,0,0.0,333.14,Jan-2016,0.0,0.0,14123.0,15100.0,1
42537,12000.0,12000.0,12000.0,36 months,0.0662,368.45,A,A2,10.0,MORTGAGE,...,0.00,0,0.0,368.45,Jan-2016,0.0,0.0,267646.0,61100.0,1
42538,12000.0,12000.0,12000.0,36 months,0.1353,407.40,B,B5,10.0,RENT,...,0.00,0,0.0,119.17,Jan-2016,0.0,15386.0,13605.0,8100.0,1
42539,15000.0,15000.0,15000.0,36 months,0.0890,476.30,A,A5,2.0,MORTGAGE,...,0.00,0,0.0,476.30,Jan-2016,0.0,1514.0,272492.0,15400.0,1
42540,12000.0,12000.0,12000.0,36 months,0.1199,398.52,B,B3,10.0,MORTGAGE,...,0.00,0,0.0,398.52,Jan-2016,0.0,0.0,327264.0,16200.0,1
42541,14000.0,14000.0,14000.0,36 months,0.1285,470.71,B,B4,4.0,RENT,...,0.00,0,0.0,470.71,Jan-2016,0.0,0.0,17672.0,4500.0,1
42542,7550.0,7550.0,7550.0,36 months,0.1624,266.34,C,C5,3.0,RENT,...,0.00,0,0.0,266.34,Jan-2016,0.0,0.0,5759.0,8000.0,1
42543,10000.0,10000.0,10000.0,36 months,0.0967,321.13,B,B1,7.0,MORTGAGE,...,0.00,0,0.0,321.13,Jan-2016,0.0,0.0,39143.0,22300.0,1
42544,3000.0,3000.0,3000.0,36 months,0.1285,100.87,B,B4,10.0,RENT,...,0.00,0,0.0,2677.23,Jan-2016,0.0,154.0,19530.0,5300.0,1


In [13]:
#loan_df = loan_df.reset_index
loan_df 

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,good_loan
42535,27050.0,27050.0,27050.0,36 months,0.1099,885.46,B,B2,10.0,OWN,...,0.00,0,0.0,885.46,Dec-2015,0.0,0.0,114834.0,59900.0,1
42536,9750.0,9750.0,9750.0,36 months,0.1398,333.14,C,C1,1.0,RENT,...,0.00,0,0.0,333.14,Jan-2016,0.0,0.0,14123.0,15100.0,1
42537,12000.0,12000.0,12000.0,36 months,0.0662,368.45,A,A2,10.0,MORTGAGE,...,0.00,0,0.0,368.45,Jan-2016,0.0,0.0,267646.0,61100.0,1
42538,12000.0,12000.0,12000.0,36 months,0.1353,407.40,B,B5,10.0,RENT,...,0.00,0,0.0,119.17,Jan-2016,0.0,15386.0,13605.0,8100.0,1
42539,15000.0,15000.0,15000.0,36 months,0.0890,476.30,A,A5,2.0,MORTGAGE,...,0.00,0,0.0,476.30,Jan-2016,0.0,1514.0,272492.0,15400.0,1
42540,12000.0,12000.0,12000.0,36 months,0.1199,398.52,B,B3,10.0,MORTGAGE,...,0.00,0,0.0,398.52,Jan-2016,0.0,0.0,327264.0,16200.0,1
42541,14000.0,14000.0,14000.0,36 months,0.1285,470.71,B,B4,4.0,RENT,...,0.00,0,0.0,470.71,Jan-2016,0.0,0.0,17672.0,4500.0,1
42542,7550.0,7550.0,7550.0,36 months,0.1624,266.34,C,C5,3.0,RENT,...,0.00,0,0.0,266.34,Jan-2016,0.0,0.0,5759.0,8000.0,1
42543,10000.0,10000.0,10000.0,36 months,0.0967,321.13,B,B1,7.0,MORTGAGE,...,0.00,0,0.0,321.13,Jan-2016,0.0,0.0,39143.0,22300.0,1
42544,3000.0,3000.0,3000.0,36 months,0.1285,100.87,B,B4,10.0,RENT,...,0.00,0,0.0,2677.23,Jan-2016,0.0,154.0,19530.0,5300.0,1


In [14]:
good_loan =  len(loan_df[(loan_df.loan_status == 'Fully Paid') |
                    (loan_df.loan_status == 'Current') | 
                    (loan_df.loan_status == 'Does not meet the credit policy. Status:Fully Paid')])
print ('Good/Bad Loan Ratio: %.2f%%'  % (good_loan/len(loan_df)*100))

Good/Bad Loan Ratio: 92.76%


In [15]:
loan_df.dtypes

loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                      float64
installment                   float64
grade                          object
sub_grade                      object
emp_length                    float64
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
revol_util                    float64
total_acc                     float64
initial_list

In [ ]:
#loan_df.to_csv("CleanLoan.csv")

## One Hot Encode 

In [16]:
columns = ['grade', 'term', 'pymnt_plan', 'home_ownership', 'purpose',  'recoveries', 'initial_list_status','verification_status']
for col in columns:
    tmp_df = pd.get_dummies(loan_df[col], prefix=col)
    loan_df = pd.concat((loan_df, tmp_df), axis=1)

In [17]:
loan_df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,purpose_small_business,purpose_vacation,purpose_wedding,recoveries_0,recoveries_1,initial_list_status_f,initial_list_status_w,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
42535,27050.0,27050.0,27050.0,36 months,0.1099,885.46,B,B2,10.0,OWN,...,0,0,0,1,0,0,1,0,0,1
42536,9750.0,9750.0,9750.0,36 months,0.1398,333.14,C,C1,1.0,RENT,...,0,0,0,1,0,1,0,1,0,0
42537,12000.0,12000.0,12000.0,36 months,0.0662,368.45,A,A2,10.0,MORTGAGE,...,0,0,0,1,0,0,1,1,0,0
42538,12000.0,12000.0,12000.0,36 months,0.1353,407.40,B,B5,10.0,RENT,...,0,0,0,1,0,0,1,0,1,0
42539,15000.0,15000.0,15000.0,36 months,0.0890,476.30,A,A5,2.0,MORTGAGE,...,0,0,0,1,0,0,1,1,0,0
42540,12000.0,12000.0,12000.0,36 months,0.1199,398.52,B,B3,10.0,MORTGAGE,...,0,0,0,1,0,1,0,0,1,0
42541,14000.0,14000.0,14000.0,36 months,0.1285,470.71,B,B4,4.0,RENT,...,0,0,0,1,0,1,0,1,0,0
42542,7550.0,7550.0,7550.0,36 months,0.1624,266.34,C,C5,3.0,RENT,...,0,0,0,1,0,0,1,1,0,0
42543,10000.0,10000.0,10000.0,36 months,0.0967,321.13,B,B1,7.0,MORTGAGE,...,0,0,0,1,0,1,0,1,0,0
42544,3000.0,3000.0,3000.0,36 months,0.1285,100.87,B,B4,10.0,RENT,...,0,0,0,1,0,1,0,0,0,1


In [18]:
loan_df.drop([
            'grade', 
            'term', 
            'pymnt_plan', 
            'home_ownership', 
            'purpose',  
            'recoveries', 
            'initial_list_status',
            'verification_status'
           ], axis=1, inplace=True)

In [19]:
loan_df.drop(['funded_amnt_inv',
              'loan_status',
              'sub_grade',
              'issue_d',
              'addr_state',
              'out_prncp',
              'earliest_cr_line',
              'out_prncp_inv',
              'total_pymnt_inv',
              'collection_recovery_fee',
              'last_credit_pull_d',
              'last_pymnt_amnt',
              'total_pymnt', 
              'total_rec_prncp', 
              'total_rec_int', 
              'total_rec_late_fee', 
              'collections_12_mths_ex_med'
             ], axis=1, inplace=True)

In [20]:
loan_df

,loan_amnt,funded_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_small_business,purpose_vacation,purpose_wedding,recoveries_0,recoveries_1,initial_list_status_f,initial_list_status_w,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
42535,27050.0,27050.0,0.1099,885.46,10.0,55000.0,22.87,0.0,0.0,14.0,...,0,0,0,1,0,0,1,0,0,1
42536,9750.0,9750.0,0.1398,333.14,1.0,26000.0,25.12,0.0,0.0,12.0,...,0,0,0,1,0,1,0,1,0,0
42537,12000.0,12000.0,0.0662,368.45,10.0,105000.0,14.05,0.0,1.0,12.0,...,0,0,0,1,0,0,1,1,0,0
42538,12000.0,12000.0,0.1353,407.40,10.0,40000.0,16.94,0.0,0.0,7.0,...,0,0,0,1,0,0,1,0,1,0
42539,15000.0,15000.0,0.0890,476.30,2.0,63000.0,16.51,0.0,0.0,8.0,...,0,0,0,1,0,0,1,1,0,0
42540,12000.0,12000.0,0.1199,398.52,10.0,130000.0,13.03,0.0,1.0,9.0,...,0,0,0,1,0,1,0,0,1,0
42541,14000.0,14000.0,0.1285,470.71,4.0,88000.0,10.02,1.0,0.0,6.0,...,0,0,0,1,0,1,0,1,0,0
42542,7550.0,7550.0,0.1624,266.34,3.0,28000.0,8.40,0.0,0.0,4.0,...,0,0,0,1,0,0,1,1,0,0
42543,10000.0,10000.0,0.0967,321.13,7.0,102000.0,15.55,2.0,0.0,9.0,...,0,0,0,1,0,1,0,1,0,0
42544,3000.0,3000.0,0.1285,100.87,10.0,25000.0,24.68,0.0,0.0,5.0,...,0,0,0,1,0,1,0,0,0,1


In [74]:
loan_df.shape

(810488, 56)

In [75]:
X_test.shape

(245002, 55)

Checking ratio

In [76]:
print("---Loan_df---")
print((loan_df.good_loan == 0).sum()) #Bad Loan
print((loan_df.good_loan == 1).sum()) #Good Loan
print(((loan_df.good_loan == 0).sum()) / (loan_df.good_loan == 1).sum()) #Ratio of Bad to Good
print("-------------")
print("---Test/Train---")
print((y_train == 0).sum()) #Bad Loan
print((y_train == 1).sum()) #Good Loan
print(((y_train == 0).sum())/(y_train == 1).sum()) #Ratio of Bad to Good

---Loan_df---
58647
751841
0.07800452489289623
-------------
---Test/Train---
45443
526228
0.08635610419818025


# Strafication And Cross Validation Through Downsampling

In [77]:
import imblearn.under_sampling

RUS = imblearn.under_sampling.RandomUnderSampler(\
                                     ratio={ 0:int(58647/10) , 1:int(751841/10)}, \
                                     random_state = 42)

X_tr_RUS, y_tr_RUS = RUS.fit_sample(X,y)


In [79]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix

logitreg0 = LogisticRegression()
parameters = {'penalty':['l1','l2'] , 'C' : [1,5,10,15], 'class_weight': ['balanced', {1 : 1, 0 : 5}, {1 : 1, 0 : 7}, {1 : 1, 0 : 9.5}, None], 'random_state': [44]}
grid0 = GridSearchCV(logitreg0 , parameters, cv=3, scoring='f1_weighted', n_jobs=1)
grid0.fit(X_tr_RUS, y_tr_RUS)
print(grid0.best_params_, grid0.best_score_)

{'C': 1, 'class_weight': None, 'penalty': 'l1', 'random_state': 44} 0.9310245500865592


In [80]:
grid0.cv_results_

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([90.29542565,  3.76453757, 29.54495104,  1.97792975, 27.08528336,
         1.18858274, 32.28705629,  3.81860391,  7.76160471,  1.10296488,
        64.32115841,  3.49962155,  4.85170539,  1.40750297,  6.32714542,
         3.75762804, 19.15030503,  6.98774203,  3.55053345,  1.03831871,
        74.90773042,  5.01708682,  5.81128971,  1.74853007,  5.66122325,
         3.32828196, 10.13412754,  5.15179928,  3.77611502,  1.16877635,
        37.93112532,  3.78183317,  4.03438854,  0.98917675,  4.82465275,
         1.32974593,  5.61485624,  5.15557504,  3.82856504,  0.93898257]),
 'std_fit_time': array([29.91032126,  0.63007326, 18.70122281,  0.99621487, 14.80353262,
         0.24777211, 12.20500485,  0.68329818,  0.42079646,  0.13090827,
        40.32407851,  0.51941587,  1.73728874,  0.33792637,  2.01172542,
         0.1204629 , 11.30750718,  1.95144685,  1.28828671,  0.09479988,
        44.89654031,  1.41825559,  0.74420139,  0.59467684,  1.28611073,
         0.51906

In [85]:
logitreg1 = LogisticRegression()
parameters = {'penalty':['l1','l2'] , 'C' : [1,4], 'class_weight': [{1 : 1, 0 : 2}, {1 : 1, 0 : 3}, None] , 'random_state': [44]}
grid1 = GridSearchCV(logitreg1 , parameters, cv=5, scoring='f1_weighted', n_jobs=1)
grid1.fit(X_tr_RUS, y_tr_RUS)
grid1.cv_results_
print(grid1.best_params_, grid1.best_score_)

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'C': 1, 'class_weight': None, 'penalty': 'l1', 'random_state': 44} 0.9309940969679141


In [86]:
grid1.cv_results_

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([13.09058127,  1.0333446 , 20.1792429 ,  2.20586581, 11.41665444,
         1.5977488 ,  6.74031777,  1.46606064,  7.79438481,  1.71983585,
         5.34520321,  1.29602852]),
 'std_fit_time': array([ 3.90560761,  0.15126242, 11.23320369,  0.74614915,  6.56851294,
         0.69293293,  2.70297794,  0.39504474,  2.49538311,  0.51092433,
         1.90373289,  0.24135847]),
 'mean_score_time': array([0.00616579, 0.00413218, 0.00790768, 0.00493116, 0.00465374,
        0.00418844, 0.00672555, 0.00617657, 0.00669022, 0.00490298,
        0.00572176, 0.0059731 ]),
 'std_score_time': array([0.00409942, 0.00037251, 0.00722152, 0.00114163, 0.00072281,
        0.00034442, 0.00319916, 0.00397549, 0.00346723, 0.0009093 ,
        0.00148767, 0.00380499]),
 'param_C': masked_array(data=[1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
         

## Train Test Split

In [36]:
from sklearn.model_selection import train_test_split

# std_scale = StandardScaler()

y = loan_df['good_loan']
X = loan_df.loc[:, loan_df.columns != 'good_loan']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)


# Modeling Without Modifications

-------------- KNNeighbors ---------------

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)


print("Metrics on Training Data")
train_predict = knn.predict(X_train)
print(metrics.accuracy_score(y_train, train_predict))
print(confusion_matrix(y_train, train_predict))

print("------------------------------")
print("------------------------------")


print("Metrics on Test Data")
y_pred = knn.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
n_neighbors = range(1,12)
knn_accuracy = list()

for n in n_neighbors:
    knn = KNeighborsClassifier(n)
    knn.fit(X_train, y_train)
    knn_prediction = knn.predict(X_test)
    knn_accuracy.append(accuracy_score(y_test, knn_prediction))

In [ ]:
plt.figure(figsize=(9,9), dpi=120)
plt.plot(n_neighbors, knn_accuracy)
plt.xlabel('# of neighbors (k)')
plt.ylabel('Accuracy on test set')
plt.title('KNN Model - Accuracy vs Neighbors')


------------------ Logistics Regression ------------------

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix

#Hardcut Linear Regression

log_model = LogisticRegression()
log_model.fit(X_train, y_train)

logistic_prediction = log_model.predict(X_test)
log_reg_accuracy = accuracy_score(y_test, logistic_prediction)

print("------------------------------")
print("Metrics on Test Data")
print (log_reg_accuracy)
print(confusion_matrix(y_test, logistic_prediction))
### You need to give it the cut-off

In [ ]:
prediction_soft = log_model.predict_proba(X_test)
confusion_matrix(y_test.values, prediction_soft)

--------------------- Support Vector   ----------------------

In [ ]:
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import svm

In [ ]:
# 1. Create a suppor vector classifier model
model_svm = svm.SVC(kernel='linear')
# 2. Fit a model on the training data
model_svm.fit(X_train, y_train)
# 3a. Predict the values using the test data
y_pred = model_svm.predict(X_test)
# 3b. Score the model
accuracy_score(y_test,y_pred)
# take a look at the predictions
confusion_matrix(y_test,y_pred)

In [ ]:
svc = LinearSVC()
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
svc = LinearSVC(C=5)
svc.fit(X_train,y_train)

y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(X_train,y_train)

y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
svc = SVC(kernel='poly')
svc.fit(X_train,y_train)


y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
svc = SVC(kernel='rbf', C=5)
svc.fit(X_train,y_train)


y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
svc = SVC(kernel='rbf', C=5, gamma=1e2) #'poly'
svc.fit(X_train,y_train)


y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
svc = SVC(kernel='rbf', C=5, gamma=.1) #'poly'
svc.fit(X_train,y_train)


y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)

In [ ]:
svc = SVC(kernel='poly', C=5)
svc.fit(X_train,y_train)


y_pred = svc.predict(X_test)
accuracy_score(y_test,y_pred)
confusion_matrix(y_test,y_pred)


--------------- DecisionTree -------------------

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=7)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

pred = dt.predict(X_test)
pred

-------------- Random Forest ----------------

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

rf_predictions = rf.predict(X_test)

#print(rf.score(X_test, y_test))
scores = cross_val_score(rf, X_test, y_test, cv=5)
print(scores)

print(sklearn.metrics.classification_report(y_test, rf_predictions))
print(sklearn.metrics.confusion_matrix(y_test, rf_predictions))

# Dealing with Imbalanced Design

In [49]:
def make_confusion_matrix(model, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    fraud_confusion = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=100)
    sns.heatmap(fraud_confusion, cmap=plt.cm.BrBG, annot=True, square=True, fmt='d',
           xticklabels=['legit', 'fraud'],
           yticklabels=['legit', 'fraud']);
    plt.xlabel('prediction')
    plt.ylabel('actual')

Experimenting with Class_Weight and C

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix

#LogisticRegression?

In [ ]:
lr_balanced = LogisticRegression(class_weight='balanced', C=5,random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


In [ ]:
print(confusion_matrix(y_test, lr_balanced.predict(X_test)))


Experimenting with Class Weight

In [ ]:
lr_9halfx = LogisticRegression(class_weight={1 : 1, 0 : 9.5}, C=3, random_state=44)
lr_9halfx.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_9halfx.predict(X_train)), roc_auc_score(y_train, lr_9halfx.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_9halfx.predict(X_test)), roc_auc_score(y_test, lr_9halfx.predict_proba(X_test)[:,1])))


In [ ]:
X_train.shape

In [ ]:
print(confusion_matrix(y_test, lr_9halfx.predict(X_test)))


In [ ]:
import pickle
pickle.dump(lr_9halfx, open("logRegCredit2.p", "wb" ))

In [ ]:
loan_df.shape

In [ ]:
lr_10x = LogisticRegression(class_weight={1 : 1, 0 : 10}, C=3, random_state=44)
lr_10x.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_10x.predict(X_train)), roc_auc_score(y_train, lr_10x.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_10x.predict(X_test)), roc_auc_score(y_test, lr_10x.predict_proba(X_test)[:,1])))


In [ ]:
print(confusion_matrix(y_test, lr_10x.predict(X_test)))


In [ ]:
lr_9x = LogisticRegression(class_weight={1 : 1, 0 : 9}, C=4, random_state=44)
lr_9x.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_9x.predict(X_train)), roc_auc_score(y_train, lr_9x.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_9x.predict(X_test)), roc_auc_score(y_test, lr_9x.predict_proba(X_test)[:,1])))
print(confusion_matrix(y_test, lr_9x.predict(X_test)))


In [ ]:
print(confusion_matrix(y_test, lr_9x.predict(X_test)))


Experimenting with L1, C, and Class_weights

In [ ]:
lr_balanced = LogisticRegression(penalty='l1', C = 2, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


In [ ]:
lr_balanced = LogisticRegression(penalty='l1', C = 5, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


In [ ]:
lr_balanced = LogisticRegression(penalty='l1', C = 20, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


Potential Proposals

In [ ]:
lr_proposal1 = LogisticRegression(penalty='l1', C = 5, class_weight={1 : 1, 0 : 9.5} ,random_state=44)

lr_proposal1.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_proposal1.predict(X_train)), roc_auc_score(y_train, lr_proposal1.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_proposal1.predict(X_test)), roc_auc_score(y_test, lr_proposal1.predict_proba(X_test)[:,1])))


In [ ]:
print(confusion_matrix(y_test, lr_proposal1.predict(X_test)))


In [ ]:
lr_proposal2 = LogisticRegression(penalty='l1', C = 3, class_weight={1 : 1, 0 : 11} ,random_state=44)

lr_proposal2.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_proposal2.predict(X_train)), roc_auc_score(y_train, lr_proposal2.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_proposal2.predict(X_test)), roc_auc_score(y_test, lr_proposal2.predict_proba(X_test)[:,1])))


In [ ]:
print(confusion_matrix(y_test, lr_proposal2.predict(X_test)))


In [ ]:
lr_proposal3 = LogisticRegression(penalty='l1', C = 3, class_weight={1 : 1, 0 : 9.75} ,random_state=44)

lr_proposal3.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_proposal3.predict(X_train)), roc_auc_score(y_train, lr_proposal3.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_proposal3.predict(X_test)), roc_auc_score(y_test, lr_proposal3.predict_proba(X_test)[:,1])))


In [ ]:
print(confusion_matrix(y_test, lr_proposal3.predict(X_test)))


In [ ]:
lr_balanced = LogisticRegression(C = 5, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


In [ ]:
lr_balanced = LogisticRegression(C = 10, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


In [ ]:
lr_balanced = LogisticRegression(C = 30, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


In [ ]:
lr_balanced = LogisticRegression(C = 100, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))


Standardize? And Other variables for GridSearch

In [ ]:
LogisticRegression?

In [ ]:
from sklearn.preprocessing import StandardScaler

ssX = StandardScaler()
X_train_scaled = ssX.fit_transform(X_train)
X_test_scaled = ssX.fit_transform(X_test)

In [ ]:
lr_balanced = LogisticRegression(C = 5, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train_scaled, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train_scaled)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train_scaled)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test_scaled)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test_scaled)[:,1])))


In [ ]:
confusion_matrix(y_test, lr_balanced.predict(X_test_scaled))

In [ ]:
lr_balanced = LogisticRegression(C = 50, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train_scaled, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train_scaled)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train_scaled)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test_scaled)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test_scaled)[:,1])))


In [ ]:
lr_balanced = LogisticRegression(C = 15, class_weight='balanced',random_state=44)

lr_balanced.fit(X_train_scaled, y_train)
print('Balanced class weights Logistic Regression Train F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_balanced.predict(X_train_scaled)), roc_auc_score(y_train, lr_balanced.predict_proba(X_train_scaled)[:,1])))
print('Balanced class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_balanced.predict(X_test_scaled)), roc_auc_score(y_test, lr_balanced.predict_proba(X_test_scaled)[:,1])))
#560

In [ ]:
confusion_matrix(y_test, lr_balanced.predict(X_test_scaled))

In [ ]:
#simple_test_case
from sklearn.model_selection import GridSearchCV

logitreg1 = LogisticRegression()
parameters = {'penalty':['l1','l2'] , 'C' : [3,3.5,4], 'class_weight': ['balanced', {1 : 1, 0 : 9.5}], 'random_state': [44]}
grid = GridSearchCV(logitreg1 , parameters, cv=2, scoring='f1_weighted', n_jobs=1)
grid.fit(X_train, y_train)

In [ ]:
grid.cv_results_

In [ ]:
print(grid.best_params_, grid.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV

logitreg0 = LogisticRegression()
parameters = {'penalty':['l1','l2'] , 'C' : [3,3.5,4], 'class_weight': ['balanced', {1 : 1, 0 : 9}, {1 : 1, 0 : 9.5}], 'random_state': [44]}
grid0 = GridSearchCV(logitreg0 , parameters, cv=3, scoring='f1_weighted', n_jobs=1)
grid0.fit(X_train, y_train)


In [ ]:
grid0.cv_results_

In [ ]:
print(grid0.best_params_, grid0.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV

logitreg2 = LogisticRegression()
parameters = {'penalty':['l1','l2'] , 'C' : [3,3.5,4], 'class_weight': ['balanced', {1 : 1, 0 : 9}, {1 : 1, 0 : 9.5}], 'random_state': [44]}
grid2 = GridSearchCV(logitreg2 , parameters, cv=3, scoring='f1_macro', n_jobs=1)
grid2.fit(X_train, y_train)

In [ ]:
grid2.cv_results_

In [ ]:
print(grid2.best_params_, grid2.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV

logitreg3 = LogisticRegression()
parameters = {'penalty':['l1','l2'] , 'C' : [3,3.5,4], 'class_weight': ['balanced', {1 : 1, 0 : 9}, {1 : 1, 0 : 9.5}, {1 : 1, 0 : 10}], 'random_state': [44]}
grid3 = GridSearchCV(logitreg3 , parameters, cv=3, scoring='f1', n_jobs=1)
grid3.fit(X_train, y_train)

In [ ]:
grid3.cv_results_

In [ ]:
print(grid3.best_params_, grid3.best_score_)

## Oversampling

In [ ]:
# randomly oversample by telling it the number of samples to have in each class
import imblearn.over_sampling
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
#from sklearn.preprocessing import StandardScaler



ROS = imblearn.over_sampling.RandomOverSampler(\
                                              ratio={0:45443*11, 1:526228}, \
                                              random_state=42)


X_tr_rs, y_tr_rs = ROS.fit_sample(X_train, y_train)



In [ ]:
X_tr_rs.shape

### A.) Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Simple Logistic Regression; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test,lr.predict(X_test)), roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])))
#print (accuracy_score(y_test, (lr.predict_proba(X_test)[:,1]))) #
#print (confusion_matrix(y_test, (lr.predict_proba(X_test)[:,1])))#


In [ ]:
confusion_matrix(y_test, lr.predict(X_test))

In [ ]:
lr_os = LogisticRegression()
lr_os.fit(X_tr_rs, y_tr_rs)
print('Logistic Regression on Oversampled Train Data; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test, lr_os.predict(X_test)), roc_auc_score(y_test, lr_os.predict_proba(X_test)[:,1])))
#print (accuracy_score(y_test, lr_os.predict_proba(X_test)[:,1])) #

#print(confusion_matrix(y_test, logistic_prediction))
#logistic_prediction = log_model.predict(X_test)

In [ ]:
confusion_matrix(y_test, lr_os.predict(X_test))

#### Hypertuning for Logistic Regression in Oversampling

In [ ]:
lr_os50 = LogisticRegression(C = 50)
lr_os50.fit(X_tr_rs, y_tr_rs)
print('Logistic Regression on Oversampled Train Data; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test, lr_os50.predict(X_test)), roc_auc_score(y_test, lr_os50.predict_proba(X_test)[:,1])))

In [ ]:
confusion_matrix(y_test, lr_os50.predict(X_test))

In [ ]:
lr_os80 = LogisticRegression(C = 80)
lr_os80.fit(X_tr_rs, y_tr_rs)
print('Logistic Regression on Oversampled Train Data; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test, lr_os80.predict(X_test)), roc_auc_score(y_test, lr_os80.predict_proba(X_test)[:,1])))

In [ ]:
confusion_matrix(y_test, lr_os80.predict(X_test))

In [ ]:
make_confusion_matrix(lr_os80) #.5

In [ ]:
lr_os150 = LogisticRegression(C = 150)
lr_os150.fit(X_tr_rs, y_tr_rs)
print('Logistic Regression on Oversampled Train Data; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test, lr_os.predict(X_test)), roc_auc_score(y_test, lr_os.predict_proba(X_test)[:,1])))

In [ ]:
confusion_matrix(y_test, lr_os150.predict(X_test))

In [ ]:
lr_os = LogisticRegression(C = 180)
lr_os.fit(X_tr_rs, y_tr_rs)
print('Logistic Regression on Oversampled Train Data; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test, lr_os.predict(X_test)), roc_auc_score(y_test, lr_os.predict_proba(X_test)[:,1])))

In [ ]:
confusion_matrix(y_test,lr_os.predict(X_test))

In [ ]:
make_confusion_matrix(lr_os) #.5

### B.) KNeighboors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_tr_rs, y_tr_rs)


print("Metrics on Training Data")
train_predict = knn.predict(X_tr_rs)
print(metrics.accuracy_score(y_tr_rs, train_predict))
print(confusion_matrix(y_tr_rs, train_predict))

print("------------------------------")
print("------------------------------")


print("Metrics on Test Data")
y_pred = knn.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
n_neighbors = range(1,12)
knn_accuracy = list()

for n in n_neighbors:
    knn = KNeighborsClassifier(n)
    knn.fit(X_tr_rs, y_tr_rs)
    knn_prediction = knn.predict(X_test)
    knn_accuracy.append(accuracy_score(y_test, knn_prediction))

In [ ]:
plt.figure(figsize=(9,9), dpi=120)
plt.plot(n_neighbors, knn_accuracy)
plt.xlabel('# of neighbors (k)')
plt.ylabel('Accuracy on test set')
plt.title('KNN Model - Accuracy vs Neighbors')


### C.) SMOTE

In [ ]:
smote = imblearn.over_sampling.SMOTE(\
                                     ratio={0:45443*11,1:526228}, \
                                              random_state=42)
    
X_tr_smote, y_tr_smote = smote.fit_sample(X_train, y_train)


#### Logistic Regression

In [ ]:
lr_smote = LogisticRegression() 
lr_smote.fit(X_tr_smote, y_tr_smote)

print('Logistic Regression on SMOTE Train Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_smote.predict(X_test)), roc_auc_score(y_test, lr_smote.predict_proba(X_test)[:,1])))


In [ ]:
lr_smote50 = LogisticRegression(C=50) 
lr_smote50.fit(X_tr_smote, y_tr_smote)

print('Logistic Regression on SMOTE Train Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_smote50.predict(X_test)), roc_auc_score(y_test, lr_smote50.predict_proba(X_test)[:,1])))


In [ ]:
lr_smote75 = LogisticRegression(C=75) 
lr_smote75.fit(X_tr_smote, y_tr_smote)

print('Logistic Regression on SMOTE Train Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_smote75.predict(X_train)), roc_auc_score(y_train, lr_smote75.predict_proba(X_train)[:,1])))
print('Logistic Regression on SMOTE Test Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_smote75.predict(X_test)), roc_auc_score(y_test, lr_smote75.predict_proba(X_test)[:,1])))


In [ ]:
make_confusion_matrix(lr_smote75) #.5

In [ ]:
lr_smote80 = LogisticRegression(C=150) 
lr_smote80.fit(X_tr_smote, y_tr_smote)

print('Logistic Regression on SMOTE Train Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_train, lr_smote80.predict(X_train)), roc_auc_score(y_train, lr_smote80.predict_proba(X_train)[:,1])))
print('Logistic Regression on SMOTE Test Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_smote80.predict(X_test)), roc_auc_score(y_test, lr_smote80.predict_proba(X_test)[:,1])))

In [ ]:
lr_smote120 = LogisticRegression(C=100) 
lr_smote120.fit(X_tr_smote, y_tr_smote)

print('Logistic Regression on SMOTE Train Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_test, lr_smote120.predict(X_test)), roc_auc_score(y_test, lr_smote120.predict_proba(X_test)[:,1])))


#### SVM 

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(k_neighbors=6, random_state=44, kind = 'svm')
X_res_train, y_res_train = sm.fit_sample(X_tr_smote, y_tr_smote)

In [ ]:
svm_sm_clf = SVC()
svm_sm_clf.fit(X_res_train, y_res_train)

svm_sm_predictions = svm_clf.predict(X_test)

#print(svm_sm_clf.score(X_test_R, y_test))
scores = cross_val_score(svm_sm_clf, X_test, y_test, cv=5)
print(scores)
#print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(sklearn.metrics.classification_report(y_test, svm_sm_predictions))
print(sklearn.metrics.confusion_matrix(y_test, svm_sm_predictions))

## Undersampling

In [ ]:
import imblearn.under_sampling

RUS = imblearn.under_sampling.RandomUnderSampler(\
                                     ratio={0:45443, 1:int(526228*.3)}, \
                                     random_state = 42)
    
X_tr_RUS, y_tr_RUS = RUS.fit_sample(X_train,y_train)


In [ ]:
lr_RUS = LogisticRegression() 
lr_RUS.fit(X_tr_RUS, y_tr_RUS)

print('Logistic Regression on Undersampled Train Data; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test,lr_RUS.predict(X_test)), roc_auc_score(y_test, lr_RUS.predict_proba(X_test)[:,1])))

In [ ]:
make_confusion_matrix(lr_RUS)

In [ ]:
lr_RUS50 = LogisticRegression(C=150) 
lr_RUS50.fit(X_tr_RUS, y_tr_RUS)

print('Logistic Regression on Undersampled Train Data; Test F1: %.3f, Test AUC: %.3f' % \
     (f1_score(y_test,lr_RUS50.predict(X_test)), roc_auc_score(y_test, lr_RUS50.predict_proba(X_test)[:,1])))

In [ ]:
make_confusion_matrix(lr_RUS50)

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import cross_val_score

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

xgb_predictions = xgb.predict(X_test)
                            
#print(xgb.score(X_test, y_test))
scores = cross_val_score(xgb, X_test, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(classification_report(y_test, xgb_predictions))
print(confusion_matrix(y_test, xgb_predictions))